<a href="https://colab.research.google.com/github/MK316/Workingpapers/blob/main/S24A-tesoldata/0622_dataprocess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# S24 TESOL data analysis (Pilot)

+ 2024.06.22~

# Part 1. Processes Step-by-Step Guide:
1. Read the Google Sheet data.
2. Fill empty cells with "No response".
3.  Rename the columns with Question numbers. (Q1~Q27)
4. Extract column names and save them to a CSV file. ('questionlist.csv')

In [ ]:
%%capture
!pip install gspread gspread-dataframe

+ [data link](https://docs.google.com/spreadsheets/d/1Bl-HkoUdmHhCnTWc6xf3SDDbQesD726-kJ4KsJQEENA/edit?usp=sharing): Need authentification

## 1, 2 & 3 Done

In [ ]:
import pandas as pd
from google.colab import auth
import gspread
from google.auth import default
from gspread_dataframe import get_as_dataframe

# Authenticate and setup
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

# Open the sheet
spreadsheet_url = 'https://docs.google.com/spreadsheets/d/1Bl-HkoUdmHhCnTWc6xf3SDDbQesD726-kJ4KsJQEENA/edit?usp=sharing'
sheet = gc.open_by_url(spreadsheet_url).sheet1

# Fetch data and convert to DataFrame
data = sheet.get_all_values()
df = pd.DataFrame(data[1:], columns=data[0])

# Debug print to check DataFrame after conversion
print("DataFrame after initial conversion:", df.head())

# Clean the DataFrame
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
df.replace('', 'No responses', inplace=True)
df.reset_index(drop=True, inplace=True)

# Final debug print
# print("DataFrame after cleaning:", df.head())


## 4. Extract column names and save them to a CSV file. ('questionlist.csv')

Column name as a dataframe (Done and saved on Google sheet datainfo file; skip this)

|QN|Questions|
|--|--|
|Q1|What's your ID?|

In [ ]:
#@markdown Record column names as csv file (done; no need to run again)
# import pandas as pd

# # Assume df is your original DataFrame and already defined
# # Example: df = pd.DataFrame(columns=['A', 'B', 'C'])

# # Get column names from the original DataFrame
# column_names = df.columns.tolist()

# # Generate identifiers from 'Q1' to 'Qn', where n is the number of columns
# qn_labels = ['Q' + str(i+1) for i in range(len(column_names))]

# # Create a new DataFrame with 'QN' and 'Questions' as the column names
# new_df = pd.DataFrame({
#     'QN': qn_labels,
#     'Questions': column_names
# })

# # Display the new DataFrame
# # new_df.to_csv("SuveryQuestions.csv",encoding='utf-8',index=False)
# # Write new_df to a CSV file
# new_df.to_csv('output.csv', index=False)  # `index=False` to avoid writing row numbers

# print(new_df)


# Part II. Data description

## [1] Column names to Q1~Q27 (df1)

+ df > df1

## [2] Question data info file (df2)
+ df2 > [Question info file](https://raw.githubusercontent.com/MK316/Workingpapers/main/S24A-tesoldata/data/S24-tesoldata-questions.csv)

In [ ]:
#@markdown Read info file (df2), change column names (df1)
import pandas as pd

# Assuming df is already defined and has 27 columns
df1 = df
# Generate new column names 'Q1', 'Q2', ..., 'Q27'
new_column_names = [f'Q{i+1}' for i in range(27)]

# Rename the columns of df
df1.columns = new_column_names

df2 = pd.read_csv('https://raw.githubusercontent.com/MK316/Workingpapers/main/S24A-tesoldata/data/S24-tesoldata-questions.csv')
# Display the DataFrame to confirm the changes
print(df1.head())
print(df2.head())


In [ ]:
# Filter the DataFrame to include only rows where 'Responsetype' is 'Likert'
filtered_df2 = df2[df2['Responsetype'] == 'Likert']

# Extract the values of 'QN' from the filtered DataFrame
filtered_qn_values = filtered_df2['QN'].tolist()
filtered_question_values = filtered_df2['Questions'].tolist()
# Display the filtered QN values
print("Filtered QN values:", filtered_qn_values)
print("Filtered Question values:", filtered_question_values)

## Demographic info summary

In [ ]:
# Convert 'Q7' to integer, coercing errors which will replace non-numeric values with NaN
df['Q7'] = pd.to_numeric(df['Q7'], errors='coerce')

In [ ]:
# Check for missing values in these columns
missing_values = df[['Q6', 'Q7', 'Q8']].isnull().sum()
print("Missing values in each column:\n", missing_values)


In [ ]:
# Describe all columns, including categorical ones
description = df['Q7'].describe(include='all')
print(description)

print("Median:", df['Q7'].median())

In [ ]:
# Frequency distribution for 'Q6'
frequency_q6 = df['Q6'].value_counts()
print("Frequency distribution for Q6:\n", frequency_q6)

# Frequency distribution for 'Q8'
frequency_q8 = df['Q8'].value_counts()
print("Frequency distribution for Q8:\n", frequency_q8)


Visualization

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(8, 6))
sns.boxplot(data=df, y='Q7')
plt.title('Boxplot of Q7')
plt.ylabel('Values of Q7')
plt.show()


In [ ]:
# Pie chart for 'Q6'
plt.figure(figsize=(4, 4))
q6_counts = df['Q6'].value_counts()
plt.pie(q6_counts, labels=q6_counts.index, autopct='%1.1f%%', startangle=90)
plt.title('Pie Chart of Q6')
plt.show()

# Pie chart for 'Q8'
plt.figure(figsize=(8, 8))
q8_counts = df['Q8'].value_counts()
plt.pie(q8_counts, labels=q8_counts.index, autopct='%1.1f%%', startangle=90)
plt.title('Pie Chart of Q8')
plt.show()


In [ ]:
# Customized Pie chart for 'Q6' with colors and exploding the largest slice
explode = (0.1 if i == q6_counts.idxmax() else 0 for i in q6_counts.index)
colors = sns.color_palette('pastel')[0:len(q6_counts)]
plt.figure(figsize=(4, 4))
plt.pie(q6_counts, labels=q6_counts.index, autopct='%1.1f%%', startangle=90, colors=colors, explode=tuple(explode))
plt.title('Pie Chart of Q6')
plt.show()


# Part III. Data exploration

## [1] Four language skills: radar plots

In [ ]:
#@markdown 1) Radar plot with Mean and STD (scale included)
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Assuming df1 is your DataFrame and it is already defined

# Define the Likert scale columns
likert_columns = ['Q13', 'Q14', 'Q15', 'Q16']

# Convert columns to numeric, coerce errors to NaN (useful if there are any non-numeric entries)
df1[likert_columns] = df1[likert_columns].apply(pd.to_numeric, errors='coerce')

# Compute the mean and standard deviation for the specified columns
mean_values = df1[likert_columns].mean()
std_values = df1[likert_columns].std()

# Print mean and standard deviation values to ensure correctness
print("Mean values of Likert scale columns:")
print(mean_values)
print("Standard deviation of Likert scale columns:")
print(std_values)

# Convert mean and standard deviation values to lists and append the first value to close the radar chart loop
mean_values_list = mean_values.tolist()
mean_values_list += mean_values_list[:1]

std_values_list = std_values.tolist()
std_values_list += std_values_list[:1]

# Define the categories
categories = ['Reading', 'Writing', 'Listening', 'Speaking']

# Number of variables
num_vars = len(categories)

# Compute angle for each category
angles = np.linspace(0, 2 * np.pi, num_vars, endpoint=False).tolist()

# The plot is made in a circular loop, so we need to "complete the loop" and append the start value to the end
angles += angles[:1]

# Plot
fig, ax = plt.subplots(figsize=(6, 6), subplot_kw=dict(polar=True))

# Plot mean values
ax.plot(angles, mean_values_list, color='blue', linewidth=2, linestyle='solid', label='Mean')

# Fill the area between mean ± standard deviation
ax.fill(angles, mean_values_list, color='blue', alpha=0.25)

# Calculate upper and lower bounds for the shaded area
upper_bound = (mean_values + std_values).tolist()
lower_bound = (mean_values - std_values).tolist()

# Close the loop for upper and lower bounds
upper_bound += upper_bound[:1]
lower_bound += lower_bound[:1]

# Fill the area between the upper and lower bounds
ax.fill_between(angles, lower_bound, upper_bound, color='blue', alpha=0.1)

# Add labels
ax.set_yticklabels([])  # Remove default y-tick labels
ax.set_xticks(angles[:-1])
ax.set_xticklabels(categories)

# Add horizontal scale numbers from 1 to 6
ax.set_ylim(0, 6)  # Set the limit for y-axis
ax.set_yticks(np.arange(1, 7))
ax.yaxis.set_tick_params(labelsize=10)
ax.set_yticklabels([str(i) for i in range(1, 7)])

# Title
plt.title('Likert Scale Radar Plot with Standard Deviation')

# Add legend
plt.legend(loc='upper right')

# Show the plot
plt.show()


+ Using median and IQR

In [ ]:
#@markdown 2) Radar plot with median and IQR (scales included 1~6)

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Assuming df1 is your DataFrame and it is already defined

# Define the Likert scale columns
likert_columns = ['Q13', 'Q14', 'Q15', 'Q16']

# Convert columns to numeric, coerce errors to NaN (useful if there are any non-numeric entries)
df1[likert_columns] = df1[likert_columns].apply(pd.to_numeric, errors='coerce')

# Compute the median and IQR for the specified columns
median_values = df1[likert_columns].median()
q1 = df1[likert_columns].quantile(0.25)
q3 = df1[likert_columns].quantile(0.75)
iqr_values = q3 - q1

# Print median and IQR values to ensure correctness
print("Median values of Likert scale columns:")
print(median_values)
print("Interquartile range (IQR) of Likert scale columns:")
print(iqr_values)

# Convert median and IQR values to lists and append the first value to close the radar chart loop
median_values_list = median_values.tolist()
median_values_list += median_values_list[:1]

iqr_values_list = iqr_values.tolist()
iqr_values_list += iqr_values_list[:1]

# Define the categories
categories = ['Reading', 'Writing', 'Listening', 'Speaking']

# Number of variables
num_vars = len(categories)

# Compute angle for each category
angles = np.linspace(0, 2 * np.pi, num_vars, endpoint=False).tolist()

# The plot is made in a circular loop, so we need to "complete the loop" and append the start value to the end
angles += angles[:1]

# Plot
fig, ax = plt.subplots(figsize=(6, 6), subplot_kw=dict(polar=True))

# Plot median values
ax.plot(angles, median_values_list, color='blue', linewidth=2, linestyle='solid', label='Median')

# Fill the area between median ± IQR/2
ax.fill(angles, median_values_list, color='blue', alpha=0.25)

# Calculate upper and lower bounds for the shaded area
upper_bound = (median_values + iqr_values / 2).tolist()
lower_bound = (median_values - iqr_values / 2).tolist()

# Close the loop for upper and lower bounds
upper_bound += upper_bound[:1]
lower_bound += lower_bound[:1]

# Fill the area between the upper and lower bounds
ax.fill_between(angles, lower_bound, upper_bound, color='blue', alpha=0.1)

# Add labels
ax.set_yticklabels([])  # Remove default y-tick labels
ax.set_xticks(angles[:-1])
ax.set_xticklabels(categories)

# Add horizontal scale numbers from 1 to 6
ax.set_ylim(0, 6)  # Set the limit for y-axis
ax.set_yticks(np.arange(1, 7))
ax.yaxis.set_tick_params(labelsize=10)
ax.set_yticklabels([str(i) for i in range(1, 7)])

# Title
plt.title('Likert Scale Radar Plot with Interquartile Range (IQR)')

# Add legend
plt.legend(loc='upper right')

# Show the plot
plt.show()


### Negative range in red color

In [ ]:
#@markdown 1) Radar plot with Mean (negative area in red)
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Assuming df1 is your DataFrame and it is already defined

# Define the Likert scale columns
likert_columns = ['Q13', 'Q14', 'Q15', 'Q16']

# Convert columns to numeric, coerce errors to NaN (useful if there are any non-numeric entries)
df1[likert_columns] = df1[likert_columns].apply(pd.to_numeric, errors='coerce')

# Compute the mean and standard deviation for the specified columns
mean_values = df1[likert_columns].mean()
std_values = df1[likert_columns].std()

# Print mean and standard deviation values to ensure correctness
print("Mean values of Likert scale columns:")
print(mean_values)
print("Standard deviation of Likert scale columns:")
print(std_values)

# Convert mean and standard deviation values to lists and append the first value to close the radar chart loop
mean_values_list = mean_values.tolist()
mean_values_list += mean_values_list[:1]

std_values_list = std_values.tolist()
std_values_list += std_values_list[:1]

# Define the categories
categories = ['Reading', 'Writing', 'Listening', 'Speaking']

# Number of variables
num_vars = len(categories)

# Compute angle for each category
angles = np.linspace(0, 2 * np.pi, num_vars, endpoint=False).tolist()

# The plot is made in a circular loop, so we need to "complete the loop" and append the start value to the end
angles += angles[:1]

# Plot
fig, ax = plt.subplots(figsize=(6, 6), subplot_kw=dict(polar=True))

# Fill the inner circle (values less than 3) with red
theta = np.linspace(0, 2 * np.pi, 100)
r = np.full_like(theta, 3)
ax.fill(theta, r, color='red', alpha=0.1, zorder=1)

# Plot mean values
ax.plot(angles, mean_values_list, color='blue', linewidth=2, linestyle='solid', label='Mean')

# Fill the area between mean ± standard deviation
ax.fill(angles, mean_values_list, color='blue', alpha=0.25, zorder=2)

# Calculate upper and lower bounds for the shaded area
upper_bound = (mean_values + std_values).tolist()
lower_bound = (mean_values - std_values).tolist()

# Close the loop for upper and lower bounds
upper_bound += upper_bound[:1]
lower_bound += lower_bound[:1]

# Fill the area between the upper and lower bounds
ax.fill_between(angles, lower_bound, upper_bound, color='blue', alpha=0.1, zorder=2)

# Add labels
ax.set_yticklabels([])
ax.set_xticks(angles[:-1])
ax.set_xticklabels(categories)

# Add horizontal scale numbers from 1 to 6
ax.set_ylim(0, 6)  # Set the limit for y-axis
ax.set_yticks(np.arange(1, 7))
ax.yaxis.set_tick_params(labelsize=10)
ax.set_yticklabels([str(i) for i in range(1, 7)])

# Title
plt.title('Likert Scale Radar Plot with Standard Deviation')

# Add legend
plt.legend(loc='upper right')

# Show the plot
plt.show()


In [ ]:
#@markdown 2) Radar plot with median (negative area in red)
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Assuming df1 is your DataFrame and it is already defined

# Define the Likert scale columns
likert_columns = ['Q13', 'Q14', 'Q15', 'Q16']

# Convert columns to numeric, coerce errors to NaN (useful if there are any non-numeric entries)
df1[likert_columns] = df1[likert_columns].apply(pd.to_numeric, errors='coerce')

# Compute the median and IQR for the specified columns
median_values = df1[likert_columns].median()
q1 = df1[likert_columns].quantile(0.25)
q3 = df1[likert_columns].quantile(0.75)
iqr_values = q3 - q1

# Print median and IQR values to ensure correctness
print("Median values of Likert scale columns:")
print(median_values)
print("Interquartile range (IQR) of Likert scale columns:")
print(iqr_values)

# Convert median and IQR values to lists and append the first value to close the radar chart loop
median_values_list = median_values.tolist()
median_values_list += median_values_list[:1]

iqr_values_list = iqr_values.tolist()
iqr_values_list += iqr_values_list[:1]

# Define the categories
categories = ['Reading', 'Writing', 'Listening', 'Speaking']

# Number of variables
num_vars = len(categories)

# Compute angle for each category
angles = np.linspace(0, 2 * np.pi, num_vars, endpoint=False).tolist()

# The plot is made in a circular loop, so we need to "complete the loop" and append the start value to the end
angles += angles[:1]

# Plot
fig, ax = plt.subplots(figsize=(6, 6), subplot_kw=dict(polar=True))

# Fill the inner circle (values less than 3) with red
theta = np.linspace(0, 2 * np.pi, 100)
r = np.full_like(theta, 3)
ax.fill(theta, r, color='red', alpha=0.1, zorder=1)

# Plot median values
ax.plot(angles, median_values_list, color='blue', linewidth=2, linestyle='solid', label='Median')

# Fill the area between median ± IQR/2
ax.fill(angles, median_values_list, color='blue', alpha=0.25, zorder=2)

# Calculate upper and lower bounds for the shaded area
upper_bound = (median_values + iqr_values / 2).tolist()
lower_bound = (median_values - iqr_values / 2).tolist()

# Close the loop for upper and lower bounds
upper_bound += upper_bound[:1]
lower_bound += lower_bound[:1]

# Fill the area between the upper and lower bounds
ax.fill_between(angles, lower_bound, upper_bound, color='blue', alpha=0.1, zorder=2)

# Add labels
ax.set_yticklabels([])  # Remove default y-tick labels
ax.set_xticks(angles[:-1])
ax.set_xticklabels(categories)

# Add horizontal scale numbers from 1 to 6
ax.set_ylim(0, 6)  # Set the limit for y-axis
ax.set_yticks(np.arange(1, 7))
ax.yaxis.set_tick_params(labelsize=10)
ax.set_yticklabels([str(i) for i in range(1, 7)])

# Title
plt.title('Likert Scale Radar Plot with Interquartile Range (IQR)')

# Add legend
plt.legend(loc='upper right')

# Show the plot
plt.show()


## [2] Before and After:

Q10, 11. How would you rate **your perspective on coding-focused digital literacy training before and after** this course?

 1 (Very negative); 6 (Very positive)

In [ ]:
import pandas as pd

# Define the Likert scale columns
likert_columns = ['Q10', 'Q11']

# Convert columns to numeric, coerce errors to NaN
df1[likert_columns] = df1[likert_columns].apply(pd.to_numeric, errors='coerce')

# Drop rows with NaN values in Q10 or Q11 to ensure the t-test works correctly
df1.dropna(subset=likert_columns, inplace=True)

### t-test (Before and After): Q10, Q11

In [ ]:
from scipy.stats import ttest_rel

# Perform paired t-test
t_stat, p_value = ttest_rel(df1['Q10'], df1['Q11'])
print(f"Paired t-test results: t-statistic = {t_stat}, p-value = {p_value}")


In [ ]:
#@markdown Data summary and plots
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import wilcoxon

# Assuming df1 is your DataFrame and it is already defined

# Define the Likert scale columns
likert_columns = ['Q10', 'Q11']

# Convert columns to numeric, coerce errors to NaN
df1[likert_columns] = df1[likert_columns].apply(pd.to_numeric, errors='coerce')

# Drop rows with NaN values in Q10 or Q11 to ensure the test works correctly
df1.dropna(subset=likert_columns, inplace=True)

# Calculate descriptive statistics
descriptive_stats = df1[likert_columns].describe()
print(descriptive_stats)

# Perform Wilcoxon signed-rank test
w_stat, p_value = wilcoxon(df1['Q10'], df1['Q11'])
print(f"Wilcoxon signed-rank test results: W-statistic = {w_stat}, p-value = {p_value}")

# Visualizations

# Box Plot
plt.figure(figsize=(8, 6))
df1[likert_columns].boxplot()
plt.title('Box Plot of Likert Scale Responses Before and After Course')
plt.xlabel('Questions')
plt.ylabel('Responses')
plt.xticks([1, 2], ['Before (Q10)', 'After (Q11)'])
plt.show()

# Paired Plot
plt.figure(figsize=(8, 6))
plt.plot(df1['Q10'], 'ro-', label='Before (Q10)')
plt.plot(df1['Q11'], 'bo-', label='After (Q11)')
plt.legend()
plt.title('Paired Plot of Responses Before and After Course')
plt.xlabel('Participants')
plt.ylabel('Responses')
plt.show()

# Radar Plot
# Calculate means
means = df1[likert_columns].mean().tolist()

# Define the categories
categories = ['Before (Q10)', 'After (Q11)']

# Number of variables
num_vars = len(categories)

# Compute angle for each category
angles = np.linspace(0, 2 * np.pi, num_vars, endpoint=False).tolist()

# The plot is made in a circular loop, so we need to "complete the loop" and append the start value to the end
means += means[:1]
angles += angles[:1]

# Plot
fig, ax = plt.subplots(figsize=(6, 6), subplot_kw=dict(polar=True))

# Plot mean values
ax.plot(angles, means, color='lightblue', linewidth=2, linestyle='solid', label='Mean Responses')
ax.fill(angles, means, color='lightblue', alpha=0.25)

# Add labels
ax.set_yticklabels([])  # Remove default y-tick labels
ax.set_xticks(angles[:-1])
ax.set_xticklabels(categories)

# Add horizontal scale numbers from 1 to 6
ax.set_ylim(0, 6)  # Set the limit for y-axis
ax.set_yticks(np.arange(1, 7))
ax.yaxis.set_tick_params(labelsize=10)
ax.set_yticklabels([str(i) for i in range(1, 7)])

# Title
plt.title('Radar Plot of Likert Scale Responses Before and After Course')

# Add legend
plt.legend(loc='upper right')

# Show the plot
plt.show()


### Paired response plot: focusing on the changes (good)


In [ ]:
#@markdown Q10, Q11 (before and after) difference plot
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Assuming df1 is your DataFrame and it is already defined

# Define the Likert scale columns
likert_columns = ['Q10', 'Q11']

# Convert columns to numeric, coerce errors to NaN
df1[likert_columns] = df1[likert_columns].apply(pd.to_numeric, errors='coerce')

# Drop rows with NaN values in Q10 or Q11 to ensure the plot works correctly
df1.dropna(subset=likert_columns, inplace=True)

# Create paired plot
plt.figure(figsize=(10, 6))

# Plot the individual points for Q10 and Q11
plt.scatter(df1.index, df1['Q10'], color='red', label='Before (Q10)')
plt.scatter(df1.index, df1['Q11'], color='blue', label='After (Q11)')

# Draw lines between the paired points to highlight changes
for i in range(len(df1)):
    plt.plot([i, i], [df1['Q10'].iloc[i], df1['Q11'].iloc[i]], color='gray', linestyle='-', linewidth=1)

# Label the axes and the plot
plt.title('Paired Plot of Responses Before and After Course Highlighting Changes')
plt.xlabel('Participants')
plt.ylabel('Responses')
plt.xticks(df1.index)
plt.yticks(np.arange(1, 7))  # Assuming Likert scale 1 to 6
plt.ylim(0.5, 6.5)  # Set limits for y-axis to improve visibility

# Add legend
# plt.legend(loc='upper left')
plt.legend(loc='lower right')

# Show the plot
plt.show()


# Adaptability and Integration (Q19, 20, 21)

In [ ]:
%%capture
!pip install pandas matplotlib seaborn

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
import pandas as pd

# Assuming df1 is already loaded
# Convert columns to integers, first replace any non-numeric characters if necessary
df1['Q19'] = pd.to_numeric(df1['Q19'], errors='coerce')
df1['Q20'] = pd.to_numeric(df1['Q20'], errors='coerce')
df1['Q21'] = pd.to_numeric(df1['Q21'], errors='coerce')

# Check for missing values that may have resulted from conversion
print(df1[['Q19', 'Q20', 'Q21']].isnull().sum())


In [ ]:
print(df1[['Q19', 'Q20', 'Q21']].describe())
print(df1[['Q19', 'Q20', 'Q21']].info())

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

fig, axes = plt.subplots(1, 3, figsize=(18, 5))
sns.histplot(df1['Q19'], kde=True, ax=axes[0], color='blue').set_title('Ease of Integration')
sns.histplot(df1['Q20'], kde=True, ax=axes[1], color='green').set_title('Alignment with Teaching Methodologies')
sns.histplot(df1['Q21'], kde=True, ax=axes[2], color='red').set_title('Benefit for Diverse Learners')
plt.tight_layout()
plt.show()


In [ ]:
sns.pairplot(df1[['Q19', 'Q20', 'Q21']], kind='scatter', diag_kind='kde', plot_kws={'alpha': 0.6})
plt.show()

Weak or no relation between questions

In [ ]:
import pandas as pd

# Assuming df is your DataFrame containing Q19, Q20, and Q21
correlation_matrix = df[['Q19', 'Q20', 'Q21']].corr()
print(correlation_matrix)


In [ ]:
corr_matrix = df1[['Q19', 'Q20', 'Q21']].corr()
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.show()


### 1. Spearman's Rank Correlation Coefficient
This is a non-parametric measure of rank correlation (statistical dependence between the rankings of two variables). It assesses how well the relationship between two variables can be described using a monotonic function. Here's how you can calculate Spearman's rank correlation in Python using pandas:

In [ ]:
import pandas as pd

# Assuming df1 is your DataFrame with the relevant data
spearman_corr = df1[['Q19', 'Q20', 'Q21']].corr(method='spearman')
print(spearman_corr)


### 2. Kendall’s Tau
Kendall’s Tau is another non-parametric measure that can be used to determine the ordinal association between two measured quantities. It is more robust to outliers than Spearman’s rank correlation. Here's how you can calculate it:

In [ ]:
kendall_corr = df1[['Q19', 'Q20', 'Q21']].corr(method='kendall')
print(kendall_corr)


### 3. Mann-Whitney U Test
If you want to compare two independent samples to determine whether they come from the same distribution, you can use the Mann-Whitney U test. This test is particularly useful if you are comparing responses from two different groups. Here’s a simple way to apply it if you had two groups to compare:

Later when classes are input in the data.

In [ ]:
from scipy.stats import mannwhitneyu

# # Example for comparing Q19 between two groups
# stat, p = mannwhitneyu(df1['Q19'][df1['Group'] == 'A'], df1['Q19'][df1['Group'] == 'B'])
# print('Statistics=%.3f, p=%.3f' % (stat, p))


### Visualization: violin plots to display the distribution
Since your data are not normally distributed, consider using box plots, violin plots, or other similar visualizations to understand the distributions and see the relationships visually:

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Boxplot for visual comparison
plt.figure(figsize=(12, 6))
sns.boxplot(data=df1[['Q19', 'Q20', 'Q21']])
plt.title('Distribution of Responses for Q19, Q20, and Q21')
plt.show()

# Violin plot for a more detailed distribution view
plt.figure(figsize=(12, 6))
sns.violinplot(data=df1[['Q19', 'Q20', 'Q21']])
plt.title('Detailed Distribution of Responses for Q19, Q20, and Q21')
plt.show()


0622 11PM